In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision
import torch.nn.functional as F
from torch import nn
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

# manage ray's relative imports
import ray
runtime_env = {"working_dir": ".." }
ray.init(runtime_env=runtime_env, dashboard_port=13065, include_dashboard=True)

from ray import tune
from ray.tune.suggest.optuna import OptunaSearch
from ray.tune import JupyterNotebookReporter

# manage beams's relative imports
import sys
sys.path.append('..')

from src.beam import beam_arguments, Experiment, Study
from src.beam import UniversalDataset, UniversalBatchSampler, PackedFolds
from src.beam import Algorithm
from src.beam import LinearNet, check_type, slice_to_index
from torchvision import transforms
import matplotlib.pyplot as plt

from src.beam import DataTensor
from src.beam.utils import is_notebook

from sklearn.datasets import fetch_covtype
import pandas as pd

from covtype_example import CovtypeDataset, CovtypeAlgorithm, RuleNet, covtype_algorithm_generator, get_covtype_parser 

2022-06-23 16:26:06,975	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:13065
2022-06-23 16:26:08,490	INFO packaging.py:388 -- Creating a file package for local directory '..'.
2022-06-23 16:26:08,509	INFO packaging.py:241 -- Pushing file package 'gcs://_ray_pkg_2adadfa5978c26c6.zip' (0.84MiB) to Ray cluster...
2022-06-23 16:26:08,519	INFO packaging.py:243 -- Successfully pushed file package 'gcs://_ray_pkg_2adadfa5978c26c6.zip'.


In [2]:
path_to_data = '/home/shared/data/dataset/covtype'
root_dir = '/home/shared/data/results/covtype'

In [3]:
args = beam_arguments(get_covtype_parser(), 
        f"--project-name=covtype --root-dir={root_dir} --algorithm=CovtypeAlgorithm --device=1 --no-half --lr-d=1e-3 --batch-size=256",
        "--n-epochs=2 --clip=0 --parallel=1 --accumulate=1 --cudnn-benchmark",
        "--weight-decay=1e-5 --beta1=0.9 --beta2=0.999", label_smoothing=.05, weight_factor=.0,
        path_to_data=path_to_data, dropout=.0, activation='gelu', channels=128, n_rules=64, n_layers=2,)

experiment = Experiment(args)

2022-06-23 15:21:57.648 | INFO     | src.beam.experiment:__init__:183 - beam project: covtype
2022-06-23 15:21:57.651 | INFO     | src.beam.experiment:__init__:184 - Experiment Hyperparameters
2022-06-23 15:21:57.653 | INFO     | src.beam.experiment:__init__:189 - project_name: covtype
2022-06-23 15:21:57.654 | INFO     | src.beam.experiment:__init__:189 - identifier: debug
2022-06-23 15:21:57.654 | INFO     | src.beam.experiment:__init__:189 - algorithm: CovtypeAlgorithm
2022-06-23 15:21:57.655 | INFO     | src.beam.experiment:__init__:189 - mp_port: None
2022-06-23 15:21:57.657 | INFO     | src.beam.experiment:__init__:189 - root_dir: /home/shared/data/results/covtype
2022-06-23 15:21:57.658 | INFO     | src.beam.experiment:__init__:189 - reload: False
2022-06-23 15:21:57.658 | INFO     | src.beam.experiment:__init__:189 - resume: -1
2022-06-23 15:21:57.659 | INFO     | src.beam.experiment:__init__:189 - override: False
2022-06-23 15:21:57.660 | INFO     | src.beam.experiment:__init_

In [4]:
# alg = covtype_algorithm_generator(experiment)

In [7]:
alg = experiment(covtype_algorithm_generator)

2022-06-23 15:35:07.080 | INFO     | src.beam.experiment:run:553 - Single worker mode
2022-06-23 15:35:07.082 | INFO     | src.beam.experiment:run_worker:45 - Worker: 1/1 is running...


train:   0%|          | 0/1360 [00:00<?, ?it/s]

validation:   0%|          | 0/452 [00:00<?, ?it/s]

2022-06-23 15:36:06.880 | INFO     | src.beam.model:step:584 - br was changed to 1
2022-06-23 15:36:06.881 | INFO     | src.beam.model:step:585 - lambda_llr was changed to 0

2022-06-23 15:36:06.884 | INFO     | src.beam.experiment:save_model_results:397 - Finished epoch 1/2:
2022-06-23 15:36:06.889 | INFO     | src.beam.experiment:log_data:443 - train:
2022-06-23 15:36:06.891 | INFO     | src.beam.experiment:log_data:447 - loss 207.968 	|
2022-06-23 15:36:06.892 | INFO     | src.beam.experiment:log_data:447 - acc 0.734467 	|
2022-06-23 15:36:06.893 | INFO     | src.beam.experiment:log_data:447 - lr 0.001 	|
2022-06-23 15:36:06.894 | INFO     | src.beam.experiment:log_data:443 - validation:
2022-06-23 15:36:06.894 | INFO     | src.beam.experiment:log_data:447 - loss 188.697 	|
2022-06-23 15:36:06.895 | INFO     | src.beam.experiment:log_data:447 - acc 0.773429 	|


train:   0%|          | 0/1360 [00:00<?, ?it/s]

validation:   0%|          | 0/452 [00:00<?, ?it/s]

2022-06-23 15:36:58.890 | INFO     | src.beam.model:step:584 - br was changed to 1
2022-06-23 15:36:58.891 | INFO     | src.beam.model:step:585 - lambda_llr was changed to 0

2022-06-23 15:36:58.895 | INFO     | src.beam.experiment:save_model_results:397 - Finished epoch 2/2:
2022-06-23 15:36:58.901 | INFO     | src.beam.experiment:log_data:443 - train:
2022-06-23 15:36:58.902 | INFO     | src.beam.experiment:log_data:447 - loss 165.038 	|
2022-06-23 15:36:58.902 | INFO     | src.beam.experiment:log_data:447 - acc 0.819548 	|
2022-06-23 15:36:58.903 | INFO     | src.beam.experiment:log_data:447 - lr 0.001 	|
2022-06-23 15:36:58.907 | INFO     | src.beam.experiment:log_data:443 - validation:
2022-06-23 15:36:58.907 | INFO     | src.beam.experiment:log_data:447 - loss 152.019 	|
2022-06-23 15:36:58.908 | INFO     | src.beam.experiment:log_data:447 - acc 0.844459 	|


In [13]:
results = alg.evaluate('test')

test:   0%|          | 0/454 [00:00<?, ?it/s]

In [15]:
np.mean(results['scalar']['acc'])

0.8330432722747063

In [14]:
%load_ext tensorboard

In [15]:
%tensorboard --logdir /home/shared/data/results/covtype/covtype/CovtypeAlgorithm/debug/0001_20220622_160022 --port=17067 --bind_all

In [6]:
args = beam_arguments(
        f"--project-name=covtype --root-dir={root_dir} --algorithm=CovtypeAlgorithm --device=1 --no-amp --lr-d=1e-3 --batch-size=256",
        "--n-epochs=100 --clip=0 --parallel=1 --accumulate=1 --cudnn-benchmark --identifier=half_precision",
        "--weight-decay=1e-5 --beta1=0.9 --beta2=0.999", label_smoothing=.05, weight_factor=.5,
        path_to_data=path_to_data, gamma=1., dropout=.0, activation='gelu', channels=512)

experiment = Experiment(args)

2022-06-23 11:44:34.677 | INFO     | src.beam.experiment:__init__:183 - beam project: covtype
2022-06-23 11:44:34.679 | INFO     | src.beam.experiment:__init__:184 - Experiment Hyperparameters
2022-06-23 11:44:34.683 | INFO     | src.beam.experiment:__init__:189 - project_name: covtype
2022-06-23 11:44:34.685 | INFO     | src.beam.experiment:__init__:189 - identifier: half_precision
2022-06-23 11:44:34.686 | INFO     | src.beam.experiment:__init__:189 - algorithm: CovtypeAlgorithm
2022-06-23 11:44:34.687 | INFO     | src.beam.experiment:__init__:189 - mp_port: None
2022-06-23 11:44:34.688 | INFO     | src.beam.experiment:__init__:189 - root_dir: /home/shared/data/results/covtype
2022-06-23 11:44:34.688 | INFO     | src.beam.experiment:__init__:189 - reload: False
2022-06-23 11:44:34.689 | INFO     | src.beam.experiment:__init__:189 - resume: -1
2022-06-23 11:44:34.690 | INFO     | src.beam.experiment:__init__:189 - override: False
2022-06-23 11:44:34.690 | INFO     | src.beam.experimen

In [7]:
alg = experiment(covtype_algorithm_generator)

2022-06-23 11:44:37.677 | INFO     | src.beam.experiment:run:553 - Single worker mode
2022-06-23 11:44:37.680 | INFO     | src.beam.experiment:run_worker:45 - Worker: 1/1 is running...


train:   0%|          | 0/1360 [00:00<?, ?it/s]

validation:   0%|          | 0/452 [00:00<?, ?it/s]


2022-06-23 11:50:32.263 | INFO     | src.beam.experiment:save_model_results:397 - Finished epoch 1/100:
2022-06-23 11:50:32.272 | INFO     | src.beam.experiment:log_data:443 - train:
2022-06-23 11:50:32.274 | INFO     | src.beam.experiment:log_data:447 - loss 228.893 	|
2022-06-23 11:50:32.274 | INFO     | src.beam.experiment:log_data:447 - acc 0.695109 	|
2022-06-23 11:50:32.275 | INFO     | src.beam.experiment:log_data:447 - lr 0.001 	|
2022-06-23 11:50:32.277 | INFO     | src.beam.experiment:log_data:443 - validation:
2022-06-23 11:50:32.283 | INFO     | src.beam.experiment:log_data:447 - loss 195.103 	|
2022-06-23 11:50:32.288 | INFO     | src.beam.experiment:log_data:447 - acc 0.758547 	|


train:   0%|          | 0/1360 [00:00<?, ?it/s]

...

train:   0%|          | 0/1360 [00:00<?, ?it/s]

validation:   0%|          | 0/452 [00:00<?, ?it/s]


2022-06-23 12:14:43.038 | INFO     | src.beam.experiment:save_model_results:397 - Finished epoch 5/100:
2022-06-23 12:14:43.057 | INFO     | src.beam.experiment:log_data:443 - train:
2022-06-23 12:14:43.058 | INFO     | src.beam.experiment:log_data:447 - loss nan 	|
2022-06-23 12:14:43.060 | INFO     | src.beam.experiment:log_data:447 - acc 0.903142 	|
2022-06-23 12:14:43.060 | INFO     | src.beam.experiment:log_data:447 - lr 0.001 	|
2022-06-23 12:14:43.062 | INFO     | src.beam.experiment:log_data:443 - validation:
2022-06-23 12:14:43.063 | INFO     | src.beam.experiment:log_data:447 - loss nan 	|
2022-06-23 12:14:43.064 | INFO     | src.beam.experiment:log_data:447 - acc 0 	|


train:   0%|          | 0/1360 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Hyperparameter optimization with ray and optuna

In [3]:
args = beam_arguments(get_covtype_parser(), 
        f"--project-name=covtype --root-dir={root_dir} --algorithm=CovtypeAlgorithm --device=0 --no-half --lr-d=1e-3 --batch-size=256",
        "--n-epochs=40 --clip=0 --parallel=1 --accumulate=1 --cudnn-benchmark",
        "--weight-decay=1e-5 --beta1=0.9 --beta2=0.999", label_smoothing=.05, weight_factor=.0,
        path_to_data=path_to_data, dropout=.0, activation='gelu', channels=128, n_rules=64, n_layers=2,)

study = Study(covtype_algorithm_generator, args)

2022-06-23 16:26:10.599 | INFO     | src.beam.experiment:__init__:82 - Hyperparameter Optimization
2022-06-23 16:26:10.600 | INFO     | src.beam.experiment:__init__:83 - beam project: covtype
2022-06-23 16:26:10.601 | INFO     | src.beam.experiment:__init__:84 - Experiment Hyperparameters
2022-06-23 16:26:10.601 | INFO     | src.beam.experiment:__init__:87 - project_name: covtype
2022-06-23 16:26:10.602 | INFO     | src.beam.experiment:__init__:87 - identifier: debug_hp_optimization_20220623_162610
2022-06-23 16:26:10.602 | INFO     | src.beam.experiment:__init__:87 - algorithm: CovtypeAlgorithm
2022-06-23 16:26:10.603 | INFO     | src.beam.experiment:__init__:87 - mp_port: None
2022-06-23 16:26:10.603 | INFO     | src.beam.experiment:__init__:87 - root_dir: /home/shared/data/results/covtype
2022-06-23 16:26:10.603 | INFO     | src.beam.experiment:__init__:87 - reload: False
2022-06-23 16:26:10.604 | INFO     | src.beam.experiment:__init__:87 - resume: -1
2022-06-23 16:26:10.604 | INFO

In [ ]:
analysis = study.tune(config={"lr_dense": tune.loguniform(1e-4, 1e-2),
                              "lr_sparse": tune.loguniform(1e-3, 1e-1),
                              "dropout": tune.uniform(.0, .25),
                              "scheduler_factor": tune.uniform(.1, .5),
                              "scheduler_patience": tune.choice([12, 16, 20]),
                              "weight_decay": tune.loguniform(1e-6, 1e-3),
                              "beta1": tune.choice([.9, .95]),
                              "beta2": tune.choice([.99, .999]),
                              "n_tables": tune.choice([1, 5, 15]),
                              "n_quantiles": tune.choice([4, 10, 20]),
                              "weight_factor": tune.choice([.0, .25, .5]),
                              "label_smoothing": tune.choice([.0, .1, .2]),
                              "channels": tune.choice([64, 128, 256]),
                              "n_rules": tune.choice([64, 128, 256]),
                              "n_layers": tune.choice([1, 2]),
                              "activation": tune.choice(['celu', 'gelu']),
                              "k_p": tune.loguniform(.01, .1),
                              "k_i": tune.loguniform(.001, .01),
                              "k_d": tune.loguniform(.001, .01),
                              "initial_mask": tune.uniform(.9, 1.),},
                       metric="mean_accuracy",
                       max_concurrent_trials=2,
                       resources_per_trial={"gpu": 1},
                       mode="max",
                       search_alg=OptunaSearch(),
                      progress_reporter=JupyterNotebookReporter(overwrite=True),
                       num_samples=1000)

Trial name,status,loc,activation,beta1,beta2,channels,dropout,initial_mask,k_d,k_i,k_p,label_smoothing,lr_dense,lr_sparse,n_layers,n_quantiles,n_rules,n_tables,scheduler_factor,scheduler_patience,weight_decay,weight_factor,acc,iter,total time (s)
runner_tune_3135cad6,RUNNING,172.17.0.7:69962,celu,0.95,0.999,256,0.0775723,0.954624,0.00257663,0.00392462,0.0312595,0,0.00216574,0.0380389,2,4,64,5,0.495696,16,2.05277e-05,0.25,0.906846,24,1913.7
runner_tune_34a16dd8,RUNNING,172.17.0.7:70000,gelu,0.9,0.999,64,0.063125,0.986053,0.00579412,0.00207835,0.0142935,0.2,0.00122931,0.0509657,2,20,256,1,0.155606,16,1.67556e-06,0.5,0.912775,7,1811.54
